In [10]:
from numpy import *
import matplotlib.pyplot as plt
from scipy import integrate
from scipy import stats
import pandas as pd
from datetime import datetime, timedelta
import os

from price import pricingModel as pm
from price import others as others
from price import optionPricingFormulas2 as opf2
from price import riskFreeRate as rate

In [11]:
def readData():
    sigmaDataPath = os.path.abspath(os.path.join("__file__", "../price/sigmaData.xls"))
    sigmaData = pd.read_excel(sigmaDataPath)
    sigmaData = sigmaData.where(sigmaData.notnull(), None)
    defaultSigma = None
    defaultTodayPrice = None ###########
    defaultQuantity = None
    
    sigmaDic = {}
    todayPriceDic = {} #todayPrice is the input file day's price and has no relation to the input T
    quantityDic = {}
    liveID = []
    
    for i in range(len(sigmaData)):
        if sigmaData["Unnamed: 1"][i] is not None:
            liveID.append(sigmaData["Unnamed: 1"][i].split()[-1])
            if sigmaData["Vol"][i] is not None:
                sigmaDic[sigmaData["Unnamed: 1"][i].split()[-1]] = (sigmaData["Vol"][i] / 100.0)
            else:
                sigmaDic[sigmaData["Unnamed: 1"][i].split()[-1]] = defaultSigma
            
            if sigmaData["Underlying_Spot"][i] is not None:
                todayPriceDic[sigmaData["Unnamed: 1"][i].split()[-1]] = sigmaData["Underlying_Spot"][i]
            else:
                todayPriceDic[sigmaData["Unnamed: 1"][i].split()[-1]] = defaultTodayPrice
                
            if sigmaData["Quantity"][i] is not None:
                quantityDic[sigmaData["Unnamed: 1"][i].split()[-1]] = sigmaData["Quantity"][i]
            else:
                quantityDic[sigmaData["Unnamed: 1"][i].split()[-1]] = defaultQuantity

    historyDataPath = os.path.abspath(os.path.join("__file__", "../price/historyData.xls"))
    data = pd.read_excel(historyDataPath)
    data = data.where(data.notnull(), None)
    data.drop([0], axis = 0, inplace = True) #inplace means data = data.changed
    data.reset_index(drop = True, inplace = True)
    data.rename(columns = {"AGGREGATION":"AGGREGATION BUNDLE"}, inplace = True)
    
    liveIDBoolSet = []
    for ID in data["AGGREGATION BUNDLE"]:
        if ID in liveID:
            liveIDBoolSet.append(True)
        else:
            liveIDBoolSet.append(False)
    data = data[liveIDBoolSet]
    data.reset_index(drop = True, inplace = True)

    
    data["k"] = data["执行价格"]
    data["startingPrice"] = data["期初价格"]
    
    s0Set = []
    for i in range(len(data)):
        if data["AGGREGATION BUNDLE"][i] in todayPriceDic:
            s0Set.append(todayPriceDic[data["AGGREGATION BUNDLE"][i]])
        else:
            s0Set.append(defaultTodayPrice)
    data["s0"] = s0Set

    sigmaSet = []
    for i in range(len(data)):
        if data["AGGREGATION BUNDLE"][i] in sigmaDic:
            sigmaSet.append(sigmaDic[data["AGGREGATION BUNDLE"][i]])
        else:
            sigmaSet.append(defaultSigma)
    data["sigma"] = sigmaSet
    
    quantitySet = []
    for i in range(len(data)):
        if data["AGGREGATION BUNDLE"][i] in quantityDic:
            quantitySet.append(quantityDic[data["AGGREGATION BUNDLE"][i]])
        else:
            quantitySet.append(defaultQuantity)
    data["quantity"] = quantitySet
    
    buySet = []
    for i in range(len(data)):
        if data["Buy/Sell"][i] == "Buy":
            buySet.append(True)
        else:
            buySet.append(False)
    data["buy"] = buySet
    
    data["price"] = None
    data["delta"] = None
    data["gamma"] = None
    data["vega"] = None
    data["theta"] = None
    return data

def setT(data, today):
    T_tmp = []
    setDay = datetime.strptime(today, '%Y-%m-%d') #fixed format
    for daysDelta in (data["期末定价日"] - setDay):
        if type(daysDelta.days) != type(int(1)): #should be type.int
            T_tmp.append(int(0))
        elif daysDelta.days < 0:
            T_tmp.append(int(0))
        else:
            T_tmp.append(daysDelta.days)
    data["T"] = T_tmp #it means from the chosen date to the end date
    return data

In [12]:
data = readData()
data = setT(data,"2019-07-08")

In [13]:
data[data["AGGREGATION BUNDLE"] == "19581693"].index

Int64Index([19], dtype='int64')

In [14]:
data.iloc[19]

AGGREGATION BUNDLE                    19581693
Product Reference     CSC2019014-3309.22.VCALL
类型                                       VCALL
Sub Type                              European
涨跌                                        CALL
欧式\美式                                 European
WIND代码                               000300.SH
结构简称                                      None
期初定价日                      2019-07-05 00:00:00
期末定价日                      2019-08-05 00:00:00
期权费支付日                     2019-08-07 00:00:00
收益兑付日                      2019-08-07 00:00:00
计划期限                                        33
执行价格                                   3309.22
上障碍                                       None
下障碍                                       None
上障碍价格                                        0
下障碍价格                                        0
Option Rebate                                0
Digital amount                            None
合约参与率                                     None
合同参与率        

In [41]:
final = datetime.strptime("2019-9-25", '%Y-%m-%d')
startDay = datetime.strptime("2019-07-08", '%Y-%m-%d')
(final - startDay).days

79

In [42]:
#ratePoints = [None, 0.0229471, 0.0250464, 0.0254817, 0.0255681, 0.0258563, 0.0263679, 0.0270334, 0.0279025, 0.0287345]
rate.getRiskFreeRate(79, ratePoints)

0.024521575

In [15]:
#computePrice(data, dataType, indexToFind, timeShift, s0IsOne, rf, dt, s0Shock, sigmaShock)
[price_tmp, delta_tmp, gamma_tmp, vega_tmp, theta_tmp] = pm.computeCallData(data, 19, 0, False, 0.0229471, 1/365, 1, 1)

In [20]:
theta_tmp

-97.38940036742503

In [21]:
price_tmp / data["startingPrice"][19]

0.12837124906080777